# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

Был получен набор данных со следующими полями:

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## 1. Откройте и изучите файл

In [8]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

data = pd.read_csv('/datasets/users_behavior.csv')

display(data.head())
print(data['is_ultra'].value_counts())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


0    2229
1     985
Name: is_ultra, dtype: int64


## 2. Разбейте данные на выборки

In [9]:
data_train, data_40 = train_test_split(data, test_size = 0.4, stratify = data['is_ultra'], random_state = 12345)
data_valid, data_test = train_test_split(data_40, test_size = 0.5, stratify = data_40['is_ultra'], random_state = 12345)


def features_target_fun(data):
    fetures = data.drop('is_ultra', axis = 1)
    target = data['is_ultra']
    return fetures, target


features_train, target_train = features_target_fun(data_train)
features_valid, target_valid = features_target_fun(data_valid)
features_test, target_test = features_target_fun(data_test)

Данные из файла были разделены на тренировочные, валидационные и тестовые в соотношении (3:1:1). Был использован параметр stratify по причине того, что количество пользователей тарифа Smart сильно отличается от количества пользователей тарифа Ultra.
Кроме этого была написана функция, которая получила признаки и целевые признаки для каждой группы данных.

## 3. Исследуйте модели

In [10]:
def decisionTreeClassifier_fun(features_train, target_train, features_valid, target_valid):
    best_model = None
    best_accuracy = 0
    best_depth = 0
    for depth in range(1, 11):
        model = DecisionTreeClassifier(max_depth = depth, random_state = 12345)
        model.fit(features_train, target_train)
        prediction = model.predict(features_valid)
        result = accuracy_score(target_valid, prediction)
        if result > best_accuracy:
            best_model = model
            best_accuracy = result
            best_depth = depth
    return best_model
    
dtc_model = decisionTreeClassifier_fun(features_train, target_train, features_valid, target_valid)

При получении результата качества модели DecisionTreeClassifier была вявлена лучшая модель с глубиной best_depth = 8 и accuracy = 0.802

In [11]:
def randomForestClassifier_fun(features_train, target_train, features_valid, target_valid):
    best_accuracy = 0
    best_depth = 0
    best_est = 0
    best_model = None
    
    # после проверки на количество деревьев в лесу (10, 51, 10)
    # был получен результат best_depth = 9 best_est = 30 : 0.8164852255054432
    # принято решение изменить цикл и исследовать модели с количством деревьев в предела [27:35]
    # потом лучший результат качества был на количестве деревьев = 27, следующий промежуток [21, 27]
    for est in range(21, 28, 1):
        for depth in range (1, 11):
            model = RandomForestClassifier(random_state=1234, n_estimators=est, max_depth=depth)
            model.fit(features_train, target_train)
            prediction = model.predict(features_valid)
            result = accuracy_score(target_valid, prediction)
            if result > best_accuracy:
                best_model = model
                best_accuracy = result
                best_depth = depth
                best_est = est
    return best_model
    

rfc_model = randomForestClassifier_fun(features_train, target_train, features_valid, target_valid)

При получении результата качества модели RandomForestClassifier была получена модель со значениями глубины best_depth = 9, количества деревьев best_est = 25 и accuracy = 0.818. В ходе выбора лучшей модели изменялись пределы цикла по количеству деревьев в лесу 1) [10, 50] 2) [27, 33] 3) [21, 27].

In [12]:
def logisticRegression_fun(features_train, target_train, features_valid, target_valid):
    model_lr = LogisticRegression(random_state=12345)
    model_lr.fit(features_train, target_train)
    prediction = model_lr.predict(features_valid)
    return model_lr

lr_model = logisticRegression_fun(features_train, target_train, features_valid, target_valid)

При получении результата качества модели LogisticRegression была получена модель со значением accuracy = 0.718

## 4. Проверьте модель на тестовой выборке

In [13]:
print('---DecisionTreeClassifier---')
print('accuracy =', dtc_model.score(features_test, target_test))
print('---RandomForestClassifier---')
print('accuracy =', rfc_model.score(features_test, target_test))
print('---LogisticRegression---')
print('accuracy =', lr_model.score(features_test, target_test))

---DecisionTreeClassifier---
accuracy = 0.7838258164852255
---RandomForestClassifier---
accuracy = 0.8133748055987559
---LogisticRegression---
accuracy = 0.7107309486780715


Лучше всех с тестовыми данными себя показала модель Случайного леса(accuracy = 0.813). Следовательно, выбираем её как оптимальную и будем использовать её в будущем.

## 5. (бонус) Проверьте модели на адекватность

In [14]:
model_dc = DummyClassifier(strategy = 'most_frequent')
model_dc.fit(features_train, target_train)

if rfc_model.score(features_test, target_test) > model_dc.score(features_test, target_test):
    print('Модель rfc_model "Случайного леса" адекватна')
else:
    print('rfc_model полный неадекват')

Модель rfc_model "Случайного леса" адекватна


После проверки модели на адекватность были получены результаты того, что Модель rfc_model "Случайного леса" адекватна. В процессе работы её сравнили с моделью DummyClassifier.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


## Вывод

В данном проектк были обучены несколько моделей и выбрана лучшая со значениями глубины best_depth = 9, количества деревьев best_est = 25 и accuracy = 0.818.
На этапе разработки были сделаны следующие выводы:
 1. Данные из файла были разделены на тренировочные, валидационные и тестовые в соотношении (3:1:1). Был использован параметр stratify по причине того, что количество пользователей тарифа Smart сильно отличается от количества пользователей тарифа Ultra. Кроме этого была написана функция, которая получила признаки и целевые признаки для каждой группы данных.
 2. При получении результата качества модели DecisionTreeClassifier была вявлена лучшая модель с глубиной best_depth = 8 и accuracy = 0.802
 3. При получении результата качества модели RandomForestClassifier была получена модель со значениями глубины best_depth = 9, количества деревьев best_est = 25 и accuracy = 0.818. В ходе выбора лучшей модели изменялись пределы цикла по количеству деревьев в лесу 1) [10, 50] 2) [27, 33] 3) [21, 27].
 4. При получении результата качества модели LogisticRegression была получена модель со значением accuracy = 0.718
 5. Лучше всех с тестовыми данными себя показала модель Случайного леса(accuracy = 0.813). Следовательно, выбираем её как оптимальную и будем использовать её в будущем.
 6. После проверки модели на адекватность были получены результаты того, что Модель rfc_model "Случайного леса" адекватна. В процессе работы её сравнили с моделью DummyClassifier.